# Merge Results

In [1]:
import json

In [2]:
def merge_inference(path):
    data = []
    for i in range(0,4):
        json_path = path + f"results_{i}.json"
        with open(json_path, "r") as f:
            data.extend(json.load(f))
            print(json_path)
    print(f"Merged Len: {len(data)}")
    output_path = path + "qwen32.json"
    with open(output_path, "w") as f:
        json.dump(data, f, indent=2)
    print(f"Results saved in: {output_path}")

In [3]:
merge_inference("./results/qwen2.5-32b/")

./results/qwen2.5-32b/results_0.json
./results/qwen2.5-32b/results_1.json
./results/qwen2.5-32b/results_2.json
./results/qwen2.5-32b/results_3.json
Merged Len: 20911
Results saved in: ./results/qwen2.5-32b/qwen32.json


In [ ]:
def create_unique_key(sample):
    return (
        sample['language'],
        sample['country'],
        sample['file_name'],
        sample['source'],
        sample['original_question_num'],
        sample['question'],
        sample['image_png']
    )

# Create sets of unique keys for both datasets
data_keys = set(create_unique_key(sample) for sample in data)
len(data_keys)

In [3]:
import json
def read_json(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

def write_json(file_path, data):
    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)
    print(f"File saved in :{file_path}")

In [4]:
file = "./results/aya-vision-32b/aya32-1024t-512i-en.json"
data = read_json(file)

In [3]:
import re
import ast
def check_if_parse(s):
    s = s.split("assistant\n")[-1]
    match = re.search(r'\{\s*"choice":\s*.*?\s*\}', s)
    if match:
        try:
            answer = ast.literal_eval(match.group())
            return True
        except Exception as e:
            return False
    return False

In [ ]:
count = 0 
for t, sample in enumerate(data):
    if not check_if_parse(sample['reasoning']):
        #print(t, sample['reasoning'])
        count += 1

NameError: name 'data' is not defined

In [ ]:
for t, sample in enumerate(data):
    if not check_if_parse(sample['reasoning']):
        if t < 1709:
            continue
        print("-"*20)
        s = sample['reasoning'].split('assistant\n')[-1]
        print(f"Index {t}: {s}")
        user_input = input("Enter a letter (A, B, C, D) or 'None' to continue: ").strip().upper()
        if user_input == 'NONE':
            continue
        elif user_input in ['A', 'B', 'C', 'D']:
            data[t]['reasoning'] = f'{{"choice": "{user_input}"}}'
        else:
            print("Invalid input. Continuing to the next item.")

NameError: name 'data' is not defined

In [ ]:
def check_if_parse(s):
    s = s.split("assistant\n")[-1]
    match = re.search(r'\{\s*"choice":\s*.*?\s*\}', s)
    
    if not match:
        # Try to capture incorrect patterns like {"choice": "B")
        match = re.search(r'(\{\s*"choice":\s*".*?"\))', s)
        if match:
            fixed_json = match.group().rstrip(')').strip() + "}"
            try:
                answer = ast.literal_eval(fixed_json)
                return True
            except Exception:
                return False
        return False
    
    try:
        answer = ast.literal_eval(match.group())
        return True
    except Exception:
        return False

In [ ]:
for sample in data:
    s = sample['reasoning']
    match = re.search(r'\{\s*"choice":\s*.*?\s*\}', s)
    if not match:
        # Try to capture incorrect patterns like {"choice": "B")
        match = re.search(r'(\{\s*"choice":\s*".*?"\))', s)
        if match:
            fixed_json = match.group().rstrip(')').strip() + "}"
            try:
                answer = ast.literal_eval(fixed_json)
                sample['reasoning'] = fixed_json
            except Exception:
                continue